# Misc. Astronomy Tools

In [16]:
%%javascript

document.head.insertAdjacentHTML('beforeend', '<style>.container { width:99% !important; }</style>');

<IPython.core.display.Javascript object>

### Stars around a coordinate

In [15]:
# Stars aroudn a coordiante, from GAIA DR2/EDR3
import astropy
from astropy.coordinates import SkyCoord, Angle
from astroquery.vizier import Vizier

compact_colnames_map = {
    "I/345/gaia2": [
        "RA_ICRS",
        "DE_ICRS",
        "RPmag",
        "Gmag",
        "BPmag",
        "Tefftemp",
        "RUWE",
        "Plx",
        "pmRA",
        "pmDE",
        "Source",                
    ],
    
    "I/350/gaiaedr3": [
        "RAJ2000",
        "DEJ2000",
        "RPmag",
        "Gmag",
        "BPmag",
        "Tefftemp",
        "RUWE",
        "sepsi",
        "epsi",
        "Plx",
        "pmRA",
        "pmDE",
        "Source",        
    ],
}

def search_nearby(ra, dec, catalog_name="I/345/gaia2", radius_arcsec=30, magnitude_limit=21, compact_columns=True):
    
    c1 = SkyCoord(ra, dec, frame="icrs", unit="deg")
    Vizier.ROW_LIMIT = -1
    columns = ["*"]
    if catalog_name == "I/350/gaiaedr3":
        columns = ["*", "epsi", "sepsi"]  # add astrometric excess noise to the output (see if a star wobbles)

    result = Vizier(columns=columns).query_region(
        c1,
        catalog=[catalog_name],
        radius=Angle(radius_arcsec, "arcsec"),
    )
    if len(result) < 1:
        return None
    result = result[catalog_name]
    if compact_columns:
        compact_colnames = compact_colnames_map.get(catalog_name)
        if compact_colnames is not None:
            # further filter to get the subset available from search result 
            compact_colnames = [c for c in compact_colnames if c in result.colnames]
            result = result[compact_colnames]    
    
#     result = result.to_pandas()
#     if result.get("Gmag") is not None:
#         result = result[result.Gmag < magnitude_limit]
#     if result.get("RUWE") is not None:
#         move_column(result, "RUWE", 7)        
#     if result.get("sepsi") is not None:
#         move_column(result, "sepsi", 8)        
#     if result.get("epsi") is not None:
#         move_column(result, "epsi", 9)        
    return result


catalog_name = "I/345/gaia2"      # https://vizier.cds.unistra.fr/viz-bin/VizieR-3?-source=I/345/gaia2
catalog_name = "I/350/gaiaedr3"   # https://vizier.cds.unistra.fr/viz-bin/VizieR-3?-source=I/350/gaiaedr3

radius_arcsec = 30

catalog_name, radius_arcsec = "B/vsx/vsx", 1200

compact_columns = True

magnitude_limit = 20
ra, dec = 263.59892769, 74.47212807

import warnings
with warnings.catch_warnings():
    # workaround https://github.com/astropy/astroquery/issues/2352
    warnings.filterwarnings("ignore", category=astropy.units.UnitsWarning)
#     warnings.filterwarnings("ignore", message=".*Unit 'e' not supported by the VOUnit standard.*")
#     warnings.simplefilter("ignore")
    result = search_nearby(ra, dec, catalog_name=catalog_name, radius_arcsec=radius_arcsec, magnitude_limit=magnitude_limit, compact_columns=compact_columns)
print(f"From {catalog_name}:")
result


From B/vsx/vsx:


OID,n_OID,Name,V,Type,l_max,max,u_max,n_max,f_min,l_min,min,u_min,n_min,l_Period,Period,u_Period,RAJ2000,DEJ2000
,,,,,,mag,,,,,mag,,,,d,,deg,deg
int32,str1,str30,uint8,str30,str1,float32,str1,str6,str1,str1,float32,str1,str6,str1,float64,str3,float64,float64
13812,B,DR Dra,0,RS,,6.550,,V,(,,0.120,,V,,26.7400000000,,263.17167,74.22736
1517608,,ASASSN-V J173604.13+743850.7,0,EA,,15.750,,V,,,16.420,,V,,5.8263700000,,264.01721,74.64741


In [48]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia

Gaia.ROW_LIMIT = 50  # Ensure the default row limit.
ra, dec = 263.59892769, 74.47212807
coord = SkyCoord(ra=ra, dec=dec, unit=(u.degree, u.degree), frame='icrs')
radius = u.Quantity(30, u.arcsec)
job = Gaia.cone_search(coord, radius)
res = job.get_results()
res

solution_id,DESIGNATION,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,astrometric_weight_al,astrometric_pseudo_colour,astrometric_pseudo_colour_error,mean_varpi_factor_al,astrometric_matched_observations,visibility_periods_used,astrometric_sigma5d_max,frame_rotator_object_type,matched_observations,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_nb_transits,rv_template_teff,rv_template_logg,rv_template_fe_h,phot_variable_flag,l,b,ecl_lon,ecl_lat,priam_flags,teff_val,teff_percentile_lower,teff_percentile_upper,a_g_val,a_g_percentile_lower,a_g_percentile_upper,e_bp_min_rp_val,e_bp_min_rp_percentile_lower,e_bp_min_rp_percentile_upper,flame_flags,radius_val,radius_percentile_lower,radius_percentile_upper,lum_val,lum_percentile_lower,lum_percentile_upper,datalink_url,dist
,,,,yr,deg,mas,deg,mas,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / mas2,1 / um,1 / um,,,,mas,,,,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,,mag,mag,mag,km / s,km / s,,K,log(cm.s**-2),dex,,deg,deg,deg,deg,,K,K,K,mag,mag,mag,mag,mag,mag,,solRad,solRad,solRad,solLum,solLum,solLum,,
int64,object,int64,int64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,int32,float32,float32,float64,float64,int16,bool,float32,float64,float64,float32,int16,int16,float32,int32,int16,bool,int32,float64,float64,float32,float32,int32,float64,float64,float32,float32,int32,float64,float64,float32,float32,float32,int16,float32,float32,float32,float64,float64,int32,float32,float32,float32,object,float64,float64,float64,float64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,float32,float32,float32,float32,float32,float32,object,float64
1635721458409799680,Gaia DR2 1655666294396195200,1655666294396195200,1358916289,2015.5,263.59890403460577,0.03170163894732282,74.47216720646641,0.04892681686518623,1.108937161662669,0.04655582626681095,23.819513,3.4416501411560043,0.04967232464463041,-21.273350405440084,0.07429961516390249,-0.2610099,-0.56073153,0.09585507,0.10948105,0.6266922,-0.32973415,-0.62051433,-0.44896287,-0.39297086,0.12271285,167,167,166,1,4.6090813,257.76434,0.0,0.0,31,True,247.27481,1.6407132332405652,0.008580896601978938,-0.11661293,19,14,0.054497305,0,22,False,193,178932.07109762123,45.150793732790945,3962.9883,12.556645,21,101483.92004431318,208.0702882847174,487.73865,12.835395,21,114246.63390573513,114.7918250967366,995.2506,12.1173115,1.2056562,0,0.7180834,0.27874947,0.43933392,-25.372872488897567,6.081422837677936,5,5500.0,3.5,-1.5,NOT_AVAILABLE,105.74822462216561,31.19368643395216,102.103825911288,81.81684407487553,100001,6141.0,5891.0,6440.0,--,--,--,--,--,--,200111,2.1014493,1.9108446,2.2835948,5.6583385,5.325697,5.99098,https://gea.esac.esa.int/data-server/datalink/links?ID=Gaia+DR2+1655666294396195200,3.429458889269471e-05
1635721458409799680,Gaia DR2 1655666294395083264,1655666294395083264,1169295133,2015.5,263.5931588177774,0.35105221587843655,74.4720050431908,0.512853972479

### Conversion between heliocentric times and barycentric time

In [17]:
# From https://gist.github.com/StuartLittlefair/4ab7bb8cf21862e250be8cb25f72bb7a

from astropy.coordinates import SkyCoord, EarthLocation
from astropy import units as u
from astropy.time import Time

def helio_to_bary(coords, hjd, obs_name):
    helio = Time(hjd, scale='utc', format='jd')
    obs = EarthLocation.of_site(obs_name)
    star = SkyCoord(coords, unit=(u.hour, u.deg)) 
    ltt = helio.light_travel_time(star, 'heliocentric', location=obs)
    guess = helio - ltt
    # if we assume guess is correct - how far is heliocentric time away from true value?
    delta = (guess + guess.light_travel_time(star, 'heliocentric', obs)).jd  - helio.jd
    # apply this correction
    guess -= delta * u.d

    ltt = guess.light_travel_time(star, 'barycentric', obs)
    return guess.tdb + ltt


def bary_to_helio(coords, bjd, obs_name):
    bary = Time(bjd, scale='tdb', format='jd')
    obs = EarthLocation.of_site(obs_name)
    star = SkyCoord(coords, unit=(u.hour, u.deg))
    ltt = bary.light_travel_time(star, 'barycentric', location=obs) 
    guess = bary - ltt
    delta = (guess + guess.light_travel_time(star, 'barycentric', obs)).jd  - bary.jd
    guess -= delta * u.d
    
    ltt = guess.light_travel    
    

### Time format conversion

In [58]:
from astropy.time import Time
import lightkurve as lk  # for BTJD format

# t1 = Time("2019-05-01", format="iso", scale="utc")
# t1 = Time("58604.4", format="mjd", scale="tdb") # BMJD, from the paper
# t1 = Time("2459307.239419309", format="jd", scale="tdb")  # BJD
# t1 = Time("2613.135", format="btjd")  # 2633, 1875.55
# t1 = Time("2611.240", format="btjd")  # 2633, 1875.55
# t1 = Time(7264.23209 + 2450000, format="jd", scale="utc")  # HJD
t1 = Time(2459692.722347, format="jd", scale="utc") 

print("Source")
print(t1.to_datetime())


t2 = t1.copy()
# t2.format = "btjd" 
t2.format="jd"
# potential issue issue: scale is still utc, can't convert to Barycentric

print("Converted:")
print(t2.to_datetime())
t2

Source
2022-04-23 05:20:10.780813
Converted:
2022-04-23 05:20:10.780813


<Time object: scale='utc' format='jd' value=2459692.722347>

### Time zone conversion

In [39]:
# adhoc time conversion helper

import pytz
from astropy.time import Time
import lightkurve as lk  # for BTJD format

# t1 = Time("2019-05-01", format="iso", scale="utc")
# t1 = Time("58604.4", format="mjd", scale="tdb") # BMJD, from the paper
t1 = Time(7264.23209 + 2450000, format="jd", scale="utc")  # HJD
# t1 = Time("2459307.239419309", format="jd", scale="tdb")  # BJD
# t1 = Time("2032.093", format="btjd")  # epoch for TIC 284361752, TOI 2294.01 (long period)
dt = t1.to_datetime()
print(dt)
dt_local = pytz.timezone('US/Pacific').fromutc(dt)
print(dt_local)

2015-08-29 17:34:12.576005
2015-08-29 10:34:12.576005-07:00


In [42]:
dt_local.hour

10

### Generate *observable* transit times based from epoch / period
- use case: to get actual local time for predicted transits

In [56]:
import pytz
from astropy.time import Time
from astropy import units as u
import numpy as np
import lightkurve as lk  # for BTJD format

title = "TIC 27064468"
labels = ["TOI 5126.01", "TOI 5126.02"]
epochs = Time([2616.125, 2577.959], format="btjd")
periods = [5.459089, 17.9098486, ] * u.day

obs_date_start = Time('2022-04-21', format='isot')
obs_date_end = Time('2022-07-01', format='isot')

# time of day deemed observable, dependent on the target and the obs_date range
obs_hr_start, obs_hr_end = 21, 3
local_time_zone = 'US/Pacific'

list_all = False  # list those in observable hours range only



def in_observable_hours_range(dt):
    return obs_hr_start <= dt.hour or dt.hour < obs_hr_end
    
    
def list_transit_times_in_range(list_all=True):
    for epoch, period, label in zip(epochs, periods, labels):
        cycle_start = int(np.ceil((obs_date_start - epoch) / period).value)
        cycle_end = int(np.ceil((obs_date_end - epoch) / period).value)  # exclusive end
        print(label) 
        
        tt_times  = [epoch + period * n for n in range(cycle_start, cycle_end)]
        for t in tt_times:
            t_val = t.value
            t_local = pytz.timezone(local_time_zone).fromutc(t.to_datetime())
            in_obs_hr_range = in_observable_hours_range(t_local)
            if list_all or in_obs_hr_range: 
                in_obs_hr_range_str = 'Y' if in_obs_hr_range else 'N'
                print(f"{in_obs_hr_range_str} \t{t_local} \t{t_val}")
    
 
print(title)
list_transit_times_in_range(list_all=list_all)

TIC 27064468
TOI 5126.01
Y 	2022-05-20 01:20:40.502400-07:00 	2719.847691
Y 	2022-05-30 23:22:51.081600-07:00 	2730.765869
Y 	2022-06-10 21:25:01.660800-07:00 	2741.684047
TOI 5126.02
